In [78]:
import os, errno, sys
from bs4 import BeautifulSoup
from requests import get
from textblob import TextBlob
import pandas as pd
from pyteaser import Summarize
from django.utils.encoding import smart_str, smart_unicode
from scraper_functions import *

proj_path = "/home/ubuntu/cv/aerial/DeepNetsEO/DeepNetsForEO/nlp/site/src/"
# This is so Django knows where to find stuff.
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "manatwee.settings")
sys.path.append(proj_path)

# This is so my local_settings.py gets loaded.
os.chdir(proj_path)

# This is so models get loaded.
from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()

from twttr.models import *
from critic.models import *


In [79]:
CHI_URL = 'http://www.chicagotribune.com/entertainment/movies/'


In [81]:
CHISoup.find_all('div',class_='trb_brk_gc_i')[0].find_all('a')[0]['href']

u'/entertainment/movies/sc-mov-mollys-game-review-1221-story.html#nt=barker&bn=Barker%2006%20-%20In%20Case%20You%20Missed%20It'

In [89]:
chi={}
for x in CHISoup.find_all('div',class_='trb_brk_gc_i'):
    title = x.find_all('a')[0].find_all('figure')[0].find_all('img')[0]['title']
    if 'review:' in title:
        r_dict = {}
        r_url = CHI_URL + x.find_all('a')[0]['href']
        title = str(title[:title.find('review')]).replace("'","").strip()
        r_soup = BeautifulSoup(get(r_url).text,'html.parser')
        r_critic = r_soup.find_all('meta',attrs={'name':'author'})[0]['content']
        r_text = ''
        for piece in r_soup.find_all('div',class_='trb_ar_page')[0].find_all('p'):
            r_text += piece.text
        r_sentiment = TextBlob(r_text).sentiment
        r_polarity = r_sentiment[0]
        r_subjectivity = r_sentiment[1]
        summary = Summarize(title,r_text)
        r_blurb = ' '.join(summary)
        r_thumbsup = 'Positive' if r_polarity >0 else 'Negative'
        print title + ": " + str(r_polarity)
        r_dict['critic'] = r_critic
        r_dict['text'] = r_text
        r_dict['blurb'] = r_blurb
        r_dict['review_subjectivity'] = r_subjectivity
        r_dict['review_polarity'] = r_polarity
        r_dict['sentiment'] = r_sentiment
        r_dict['thumbsup'] = r_thumbsup
        chi[title]=r_dict

Mollys Game: 0.0643122400475
Pitch Perfect 3: 0.161354312354
Downsizing: 0.0414239332096
Father Figures: 0.100643681715
All the Money in the World: 0.0874132166738
I, Tonya: 0.127602741828
Jumanji: Welcome to the Jungle: 0.154574420677
The Ballad of Lefty Brown: 0.156209270121
Ferdinand: 0.120436507937
Call Me by Your Name: 0.204645999162
Star Wars: The Last Jedi: 0.169744716068
Una: 0.0819171522922
Wonder Wheel: 0.133166104141
The Shape of Water: 0.109996269513
Darkest Hour: 0.128486673969
For Ahkeem: -0.0033411033411
Thirst Street: 0.0926622957146
The Disaster Artist: 0.10334202383
Bill Nye: Science Guy: 0.112206959707
Gilbert: 0.13053268701
The Future Perfect: 0.154458536255
Sylvio: 0.14806041181
Coco: 0.144967532468
Roman J. Israel, Esq.: 0.0815493646139
BPM: 0.144071180556


In [38]:
a=CHISoup.find_all('div',class_='trb_brk_gc_i')[0].find_all('a')[0].find_all('figure')[0].find_all('img')[0]['title']

In [45]:
str(a[:a.find('review')]).replace("'","").strip()

'Mollys Game'

In [50]:
molly_url = 'http://www.chicagotribune.com/entertainment/movies//entertainment/movies/sc-mov-mollys-game-review-1221-story.html#nt=barker&bn=Barker%2006%20-%20In%20Case%20You%20Missed%20It'
molly_soup = BeautifulSoup(get(molly_url).text,'html.parser')

In [54]:
molly_soup.find_all('meta',attrs={'name':'author'})[0]['content']

u'Michael Phillips'

In [69]:
r_text = ''
for piece in molly_soup.find_all('div',class_='trb_ar_page')[0].find_all('p'):
    r_text += piece.text

In [70]:
r_text

u'Molly Bloom\u2019s 2014 memoir \u201cMolly\u2019s Game\u201d was more of a tell-some than a tell-all. In the book, the former freestyle skiing Olympic hopeful discussed the accident that derailed her athletic career. Mainly, she wrote about her improbable career running a pricey, underground poker game in Los Angeles and, later, in New York City, where she ran afoul of mobsters, drugs and the feds, who arrested Bloom as part of a mafia investigation.Her book named names, up to a point. Leonardo DiCaprio, Ben Affleck and Tobey Maguire were among her A-list regulars, blowing through cash like Kleenex. But her memoir left a lot out, and that\u2019s where writer-director Aaron Sorkin\u2019s movie \u201cMolly\u2019s Game,\u201d taking place before and after the publication of her book, comes in.It\u2019s a good, brash biopic. For the first hour it\u2019s very nearly terrific. Jessica Chastain plays Molly, driven hard by her taskmaster father (Kevin Costner), growing up in a fiercely compe

In [95]:
Critic.objects.all()

<QuerySet [<Critic: Glenn Kenny>, <Critic: Andy Webster>, <Critic: Ben Kenigsberg>, <Critic: Manohla Dargis>, <Critic: Jeannette Catsoulis>, <Critic: Teo Bugbee>, <Critic: Ken Jaworowski>, <Critic: Monica Castillo>, <Critic: A.O. Scott>, <Critic: Neil Young >, <Critic: Stephen Dalton>, <Critic: Todd McCarthy>, <Critic: John DeFore>, <Critic: Frank Scheck>, <Critic: Sheri Linden >, <Critic: Justin Lowe >, <Critic: Michael Rechtshaffen>, <Critic: Jonathan Holland >, <Critic: Jordan Mintzer>, <Critic: Stephen Farber >, '...(remaining elements truncated)...']>

In [93]:
Critic.objects.get(name='Guy Lodge')

<Critic: Guy Lodge>

In [101]:
critic_score=0
pos_revs=0
total_revs=0
for item in Reviews.objects.filter(movie=Movie.objects.get(name='The Post')):
    j+=1
    if item.thumbsup == 'Positive':
        i+=1
critic_score = float(i)/float(j)
    

In [102]:
critic_score

1.0

In [103]:
i

4

In [104]:
j

4